In [2]:
from groq import Groq

In [3]:

client = Groq(api_key="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")

In [4]:
def explain_claim_with_llm(
    claim_id, provider_id, fraud_prob, predicted_label,
    tp, fp, fn, avg_claim_value,
    personnel_cost, infra_cost, compliance_cost,
    reason_from_model=None
):
    explanation_prompt = f"""
    You are an AI fraud auditor. Explain why this claim is predicted as {predicted_label}.

    Claim ID: {claim_id}
    Provider ID: {provider_id}
    Fraud Probability: {fraud_prob:.2f}
    Prediction: {predicted_label}

    Reason from model: {reason_from_model if reason_from_model else "Provide a general reason based on fraud probability and claim context."}

    Instructions:
    - Give a short, simple explanation in plain language.
    - If prediction is FRAUD → explain why it seems suspicious.
    - If prediction is NON-FRAUD → explain why it seems normal.
    """
    table_prompt = f"""
    Prepare a neat table with all metrics and costs.

    Claim ID: {claim_id}
    Provider ID: {provider_id}

    Metrics:
    - True Positives: {tp}
    - False Positives: {fp}
    - False Negatives: {fn}
    - Avg Claim Value: ${avg_claim_value:,.2f}

    Operational Costs (estimated):
    - Personnel: ${personnel_cost}
    - Infrastructure: ${infra_cost}
    - Compliance: ${compliance_cost}
    """

    explanation_response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": explanation_prompt}],
        temperature=0
    )
    explanation = explanation_response.choices[0].message.content.strip()
    table_response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": table_prompt}],
        temperature=0
    )
    table_output = table_response.choices[0].message.content.strip()

    return explanation, table_output


In [5]:
claim_id = "CLM123"
provider_id = "PRV456"
fraud_prob = 0.87
predicted_label = "FRAUD"
tp, fp, fn = 95, 22, 15
avg_claim_value = 1800.0
personnel_cost = 5000
infra_cost = 1500
compliance_cost = 800
reason_from_model = "The claim is unusually expensive compared to similar treatments in the same region."

In [6]:
explanation, table_output = explain_claim_with_llm(
    claim_id, provider_id, fraud_prob, predicted_label,
    tp, fp, fn, avg_claim_value,
    personnel_cost, infra_cost, compliance_cost,
    reason_from_model
)

print("=== Explanation ===")
print(explanation)
print("\n=== Metrics & Costs Table ===")
print(table_output)

=== Explanation ===
This claim (CLM123) is predicted as FRAUD because it appears to be much more expensive than similar treatments in the same area. This unusual cost raises suspicions, suggesting that the claim might be inflated or fake.

=== Metrics & Costs Table ===
Here is the neat table with all metrics and costs:

| **Category** | **Metric/Cost** | **Value** |
| --- | --- | --- |
| **Metrics** | True Positives | 95 |
|  | False Positives | 22 |
|  | False Negatives | 15 |
|  | Avg Claim Value | $1,800.00 |
| **Operational Costs** | Personnel | $5,000 |
|  | Infrastructure | $1,500 |
|  | Compliance | $800 |
|  | **Total Operational Costs** | **$7,300** |

Note: I added a "Total Operational Costs" row to calculate the sum of all operational costs. Let me know if you'd like me to add anything else! 

Also, for reference, here are the claim details:
- Claim ID: CLM123
- Provider ID: PRV456
